<a href="https://colab.research.google.com/github/ShashwatVv/DetectingSaracasm-In-NewsHeadlines/blob/main/Detecting_Sarcasm_in_News_headlines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###  Let's first load the dataset. 

In [8]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json \
    -O /tmp/sarcasm.json

--2022-03-28 10:12:08--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.218.128, 108.177.11.128, 74.125.31.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.218.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘/tmp/sarcasm.json’

/tmp/sarcasm.json   100%[===================>]   5.38M  --.-KB/s    in 0.02s   

2022-03-28 10:12:08 (218 MB/s) - ‘/tmp/sarcasm.json’ saved [5643545/5643545]



In [5]:
##  The libraries to be imported
import numpy  as np
import json 
import io
import tensorflow  as  tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer  as TKN
from tensorflow.keras.preprocessing.sequence import pad_sequences as padseq
print("Imported!!")

Imported!!


In [10]:
## Let's load the json-data

file_sarc = open('/tmp/sarcasm.json', 'r')
data = json.load(file_sarc)
file_sarc.close()

In [20]:
info = data[0].keys()
print("The 3 features associated with the news headlines are:", *info)

The 3 features associated with the news headlines are: article_link headline is_sarcastic


In [21]:
## let's create separate list of sentences, labels and  links/urls 
list_of_labels = list() ##sarcastic or non sarcastic
list_of_links = list() ##url to the article 
list_of_sentences = list() 

for instance in data:
  list_of_labels.append(instance['is_sarcastic'])
  list_of_links.append(instance['article_link'])
  list_of_sentences.append(instance['headline'])

print("Length of each of these lists are:",  len(list_of_labels), len(list_of_links), len(list_of_sentences))

Length of each of these lists are: 26709 26709 26709


In [22]:
## Let's do  some standard preprocessing!!
##create  a  tokenizer instance
tokenizer_instance = TKN(oov_token = '<UKN>')
##let's fit this object on the sentence lists
tokenizer_instance.fit_on_texts(list_of_sentences)
word_index = tokenizer_instance.word_index

In [23]:
word_index

{'<UKN>': 1,
 'to': 2,
 'of': 3,
 'the': 4,
 'in': 5,
 'for': 6,
 'a': 7,
 'on': 8,
 'and': 9,
 'with': 10,
 'is': 11,
 'new': 12,
 'trump': 13,
 'man': 14,
 'from': 15,
 'at': 16,
 'about': 17,
 'you': 18,
 'this': 19,
 'by': 20,
 'after': 21,
 'up': 22,
 'out': 23,
 'be': 24,
 'how': 25,
 'as': 26,
 'it': 27,
 'that': 28,
 'not': 29,
 'are': 30,
 'your': 31,
 'his': 32,
 'what': 33,
 'he': 34,
 'all': 35,
 'just': 36,
 'who': 37,
 'has': 38,
 'will': 39,
 'more': 40,
 'one': 41,
 'into': 42,
 'report': 43,
 'year': 44,
 'why': 45,
 'have': 46,
 'area': 47,
 'over': 48,
 'donald': 49,
 'u': 50,
 'day': 51,
 'says': 52,
 's': 53,
 'can': 54,
 'first': 55,
 'woman': 56,
 'time': 57,
 'like': 58,
 'her': 59,
 "trump's": 60,
 'old': 61,
 'no': 62,
 'get': 63,
 'off': 64,
 'an': 65,
 'life': 66,
 'people': 67,
 'obama': 68,
 'now': 69,
 'house': 70,
 'still': 71,
 "'": 72,
 'women': 73,
 'make': 74,
 'was': 75,
 'than': 76,
 'white': 77,
 'back': 78,
 'my': 79,
 'i': 80,
 'clinton': 81,
 '